# Laboratorio 2

- Brando Reyes
- Juan Pablo Solis

## Task 1 - Preguntas Teoricas

1. ¿Por qué el modelo de Naive Bayes se le considera "naive"?
    - Este modelo se considera "Naive" debido a que asume que todas las características son independientes entre sí, lo cual rara vez es cierto en problemas del mundo real
2. Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo?
3. Investigue sobre Random Forest y responda:
    -  ¿Qué tipo de ensemble learning es este modelo?
        - Random Forest es un modelo de bagging el cual combina múltiples modelos que se entrenan de manera independiente utilizando diferentes subconjuntos de datos, y luego promedia o vota para obtener la predicción final.
    - ¿Cuál es la idea general detrás de Random Forest?
        - La idea general de Random Forest es combinar múltiples árboles de decisión para mejorar la precisión y reducir el riesgo de sobreajuste. 
    - ¿Por qué se busca baja correlación entre los árboles de Random Forest?
        - La baja correlación entre los árboles es clave para el éxito de Random Forest porque aumenta su capacidad de generalización. Si los árboles son altamente correlacionados, cometerán los mismos errores, lo que limitaría la mejora obtenida al combinarlos